In [243]:
import glob
import pandas as pd
import re
import os

In [287]:
chars = ['순재', '문희', '해미', '준하', '민용', '신지', '민정', '교감', '윤호', '민호']

def isTalk(a_string):
    p1 = len(re.findall(r'^\w+\t', a_string))
    p2 = len(re.findall(r'^\w+\s+[()]', a_string))
    p3 = len(re.findall(r'^\t\n', a_string))
    first_word = a_string.split(" ")[0]
    p4 = int(first_word in chars)
    res = p1 + p2 + p4
    if res > 0:
        return True
    else:
        return False

In [288]:
def extract_person(a_string):
    if len(re.findall(r'^\w+\t', a_string)) > 0:
        person = a_string.split("\t")[0]
        script = " ".join(a_string.split("\t")[1:])
    elif len(re.findall(r'^\w+\s+[()]', a_string)) > 0:
        person = a_string.split(" ")[0]
        script = " ".join(a_string.split(" ")[1:])
    else:
        person = "delete"
        script = a_string
    return person, script

In [289]:
def preprocess_script(file_path):
    with open(file_path, "r", encoding="cp949") as f:
        data = f.readlines()
    df = pd.DataFrame(data)
    df.columns = ['script']
    df = df[df.script != "\n"]
    
    df['isTalk'] = df.script.map(isTalk)
    df['first_char'] = df.script.map(lambda x: x[0])
    df = df[df.first_char != "씬"].reset_index(drop=True)
    
    df['counter'] = 0
    counter = 0
    for i in range(0, len(df)):
        current_is_talk = df.ix[i]['isTalk']
        if (current_is_talk == True):
            counter = counter + 1
        df.set_value(i, 'counter', counter)
        
    df2 = pd.DataFrame(df.groupby('counter').script.apply(lambda x: "".join(list(x))))
    
    df2.reset_index(drop=True, inplace=True)
    
    df2['person'], df2['script_re'] = zip(*df2.script.map(extract_person))
    df2['script_re2'] = df2.script_re.map(lambda x: x.replace("\n\t", " ").replace("\n", "").replace("\t", ""))
    
    df3 = df2[df2.person != "delete"][['person', 'script_re2']]
    df3['person'] = df3['person'].map(lambda x: x.strip())
    df3['script_re2'] = df3['script_re2'].map(lambda x: re.sub('^\s+', "", x))
    df3['final_script'] = df3.apply(lambda row: row.person + ": " + row.script_re2, axis=1)
    return df3.final_script.tolist()

In [290]:
files = glob.glob("./data/*.txt")

In [298]:
temp = [preprocess_script(os.path.join(file)) for file in files]

In [299]:
script_list = [sentence for sublist in temp for sentence in sublist]

In [297]:
script_list[100: 150]

['개성댁: 어머! (벌떡 일어나며) 어유 안녕하세요.',
 '순재: 아 네.',
 '개성댁: 나 갈께~ 안녕히 계세요. (후다닥 나가는)',
 '문희: 가. 당신 이 시간에 왠일이유?',
 '순재: (서류 집으며) 안돼 이쪽에 걸어 (나가다) 머리는 뭘그런걸 쓰고 있어? (벗겨보는)',
 '문희: 어~ (벗겨지면 파마용 롤이 붙어있다) 개성댁이랑 미장원에서 머리 말다 잠깐 온거야.',
 '순재: 파마를 왜? 설마 뽀글뽀글한 뽀글파마 그거 하는거야?',
 '문희: 예.',
 '순재: (화내는) 아 내가 싫어하는 거 알면서 왜 뽀글파마를해?',
 '문희: 그게 젤 편해. 한번 하면 만사 편한걸.',
 '순재: 그래도 내가 싫어하는데 왜 해! 나랑 상의도 없이! 아니 오늘따라 아들놈이나 할망구나 왜 이렇게 상의도 없이 맘대로 헛짓꺼리들이야?',
 '문희: 다시 풀어달라 그래요? 풀어도 돈은 다 내야 돼..',
 '순재: 아우 씨! (화가 나서 나가는)  수업 마친 학생들 삼삼오오 걸어오고 윤호 걸어오는데 핸드폰 울리는',
 '윤호: 예 찬성과 승현, 옥상에 있다. 승현은 폼 잡으며 각 세우고 서 있고 찬성이 전화하는',
 '찬성: 야 이윤호 나 황찬성이다.',
 '윤호: 그래서?찬성          승현이가 우리 전학온 기념으로 함 뜨자고 기다리고있는데 왜 코빼기를 안보여? 겁나나보지?',
 '윤호: (픽 웃으며) 그래 겁난다 아우 무서워~',
 '찬성: 야 우리 지금 옥상에 있거든? 쉬끼 10초안에 튀어 와라.',
 '윤호: (꺼버리고 가는)',
 '찬성: 여보세요? 야 이윤호! 이 쉬끼가 진짜! (핸드폰 끊으며) 끊어버리는데?',
 '승현: (눈썹 찌푸리며) 뭐?!! (하고) 그럼 모시러 가야지. (가는) 승현과 찬성이 거칠게 들어온다. 청소하던 아이들 겁에 질려 비켜서고 승현 1학년 4반문 거칠게 열어젖힌다. 승현과 찬성이 들어온다.',
 '찬성: (문 확 열고) 이윤호! 텅 빈 교실에 민호 혼자 앉아 공부하고 있다.',
 '찬성: 뭐야 다 갔네? (민호 보